<a href="https://colab.research.google.com/github/claredavies/ArtificialIntelligience/blob/main/ComputationalIntelligienceAssignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [1]:
import numpy
import pandas as pd
import random

In [2]:
from numpy.random import randint
from numpy.random import rand

# Inputs

Class Info

In [104]:
#Creating Series
class_series = pd.Series(['Class1', 'Class1', 'Class1', 'Class1','Class2', 'Class2', 'Class2', 'Class2'])
subject_series = pd.Series(['Maths', 'English', 'Chemistry', 'History','Maths', 'English', 'History', 'Physics'])
hours_series = pd.Series([3, 4, 5, 3, 2, 4, 5, 3])


#Creating Dictionary
frame = { 'Class': class_series, 'Subject': subject_series, 'Hours': hours_series }

#Creating Dataframe
class_subject_hours_dataframe = pd.DataFrame(frame)

#Printing dataframe
print(class_subject_hours_dataframe)

    Class    Subject  Hours
0  Class1      Maths      3
1  Class1    English      4
2  Class1  Chemistry      5
3  Class1    History      3
4  Class2      Maths      2
5  Class2    English      4
6  Class2    History      5
7  Class2    Physics      3


Define Timetable

In [105]:
startTime = 10
durationLesson = 1
lunchTime = 12
durationLunch = 1
endTime = 15

singleTimetable = []
daysOfWeek = ['Mon','Tue','Wed', 'Thur']

for day in daysOfWeek:
  for time in range(startTime, endTime, durationLesson):
    if time != lunchTime:
      singleTimetable.append([day, time])

print(singleTimetable)

[['Mon', 10], ['Mon', 11], ['Mon', 13], ['Mon', 14], ['Tue', 10], ['Tue', 11], ['Tue', 13], ['Tue', 14], ['Wed', 10], ['Wed', 11], ['Wed', 13], ['Wed', 14], ['Thur', 10], ['Thur', 11], ['Thur', 13], ['Thur', 14]]


Teacher Info

In [144]:
teacher_series = pd.Series(['ProfessorA', 'ProfessorA', 'ProfessorB', 'ProfessorB','ProfessorC', 'ProfessorC'])
subject_series = pd.Series(['Maths', 'English', 'Chemistry', 'History','Maths', 'Physics'])

#Creating Dictionary
frame = { 'Teacher': teacher_series, 'Subject': subject_series}

#Creating Dataframe
teacher_subject_dataframe = pd.DataFrame(frame)

teacher_subject_array = teacher_subject_dataframe.to_numpy()
#Printing dataframe
print(teacher_subject_array)

# dictionary generated for convenience
teachers_dict = dict((t, {
                        'Avaliability' : [True]*len(singleTimetable),  # Maybe is just enought with the number of hours assignable per day
                        'Subjects'      : teacher_subject_dataframe.groupby('Teacher').get_group(str(t))['Subject'].tolist()
                        }
                    ) for t in teacher_subject_dataframe.Teacher.unique())

print(teachers_dict)

[['ProfessorA' 'Maths']
 ['ProfessorA' 'English']
 ['ProfessorB' 'Chemistry']
 ['ProfessorB' 'History']
 ['ProfessorC' 'Maths']
 ['ProfessorC' 'Physics']]
{'ProfessorA': {'Avaliability': [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True], 'Subjects': ['Maths', 'English']}, 'ProfessorB': {'Avaliability': [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True], 'Subjects': ['Chemistry', 'History']}, 'ProfessorC': {'Avaliability': [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True], 'Subjects': ['Maths', 'Physics']}}


In [50]:
def create_timetables(class_subject_hours_dataframe, timetable) -> numpy.ndarray:
  """
    Generates an array of blank timetables ready to be assigned, will have one row for each different class and n
    columns depending of the slots avaliable per week 
  """ 
  all_timetables = []
  blank_timetable = [None] * len(timetable)
  number_class_timetables = class_subject_hours_dataframe.Class.nunique()  # (Class 1, Class 2, ...)

  for x in range(number_class_timetables):
    all_timetables.append(blank_timetable)

  # print(all_timetables)
  all_timetables_array = numpy.array(all_timetables)
  return all_timetables_array

In [145]:
def make_array_1_hour_class(class_mask):
  """
  Generates a list with the subject names for convenience

  if the mask is

          Class    Subject  Hours
    0  Class1      Maths      3
    1  Class1    English      4
    2  Class1  Chemistry      5
    3  Class1    History      3
  
  The output will be

  ['Maths', 'Maths', 'Maths', 'English', 'English', 'English', 'English', 'Chemistry', 'Chemistry', 'Chemistry', 'Chemistry', 'Chemistry', 'History', 'History', 'History']
  """
  list_classes_1hr = []
  for index, row in class_mask.iterrows():
    number_hours_per_class = row['Hours']
    for x in range(number_hours_per_class):
      list_classes_1hr.append(row['Subject'])
  list_finished = list_classes_1hr.copy()
  return list_finished

def assign_teacher(subject:str, teacher_subject_dataframe):
  """ 
      Return a possible teacher name that teaches that subject, 
      in case of having more than one capable teacher, it will be
      selected randomly.

      TODO: Could be added a constrain of hours for each teacher
  """
  candidates = []
  for t in teacher_subject_dataframe.Teacher.unique():
    if subject in teachers_dict[t]['Subjects']:
      candidates.append(t)

  if len(candidates)==0:
    return 'Non-Possible-Assignation'
  elif len(candidates)==1:
    return candidates[0]
  else:
    return random.choice(candidates)

# Genetic Algorithms Functions

In [130]:
def generate_intial_population(class_subject_hours_dataframe, teacher_subject_dataframe, timetable, populationSize):
  population = []
  class_names = class_subject_hours_dataframe.Class.unique()
  number_class_names = class_subject_hours_dataframe.Class.nunique()

  for a in range(populationSize):
    trackClass = 0
    all_timetables = create_timetables(class_subject_hours_dataframe, timetable)
    for class_name in class_names:
      mask = class_subject_hours_dataframe.loc[class_subject_hours_dataframe['Class'] == class_name]
      list_classes_1hr = make_array_1_hour_class(mask)

      list_indices_single_timetable = [None] * len(timetable)

      if(len(list_classes_1hr) > len(list_indices_single_timetable) ):
        print("Not enough hours in timetable for class hours set!!")
        return

      for i in range(len(list_indices_single_timetable)):
        list_indices_single_timetable[i] = i
    
      range_end_single_timetable = len(list_indices_single_timetable) - 1

      for j in range((len(list_classes_1hr) - 1)):
        random_num = random.randint(0, range_end_single_timetable)

        random_num_index_value = list_indices_single_timetable[random_num]

        all_timetables[trackClass][random_num_index_value] = (list_classes_1hr[j], assign_teacher(subject=list_classes_1hr[j], teacher_subject_dataframe=teacher_subject_dataframe))
    
        list_indices_single_timetable.pop(random_num)
        range_end_single_timetable = range_end_single_timetable - 1
    
      trackClass = trackClass + 1
    
    population.append(all_timetables)
  return population

In [186]:
# CHECK: We have to check as 2 different clashes if we have same subject the same hour
# or same profesor the same hour in different clases?

def calClashes(pop):
  num_timetable_slots = pop.shape[1]
  num_classes = pop.shape[0]
  clash = 0
  for i in range(num_timetable_slots):
    for j in range(1, num_classes):
      # Original function here
      # if(pop[(j-1)][i] == pop[j][i]):
      #   if(pop[(j-1)][i] != None):
      #    clash = clash + 1
      if(pop[(j-1)][i] != None and pop[(j)][i] != None):  # If we are not dealing with empty spaces
        if(pop[(j-1)][i] == pop[j][i]) or pop[(j-1)][i][1] == pop[j][i][1]: # check if we have the same class at the same time or one teacher is assigned in two classes at the same time
          clash = clash + 1
  return clash

In [183]:
def fitnessFunction(pop):
  arrPop = numpy.array(pop)
  num_representations = arrPop.shape[0]
  NoClashesList = [None] * num_representations

  # print("no representations:  " + str(num_representations))
  for i in range(num_representations):
    populationSelect = pop[i]
    noClashes = calClashes(populationSelect)
    NoClashesList[i] = noClashes

  return NoClashesList

In [184]:
def recombine(parent1, parent2):
  parent1Array = numpy.array(parent1)
  parent2Array = numpy.array(parent2)

  recombinationResults = []

  noTimetables = parent2Array.shape[0]
  crossOverIndex = int(parent1Array.shape[1]/2)

  recombinedParent1Array = parent1Array.copy()
  recombinedParent2Array = parent2Array.copy()

  for i in range(noTimetables): 
    classRecombinationResults = []  
    parent1FirstHalf = parent1Array[i][0:(crossOverIndex-1)]
    parent1SecondHalf = parent1Array[i][(crossOverIndex-1):(parent1.shape[1])]
    parent2FirstHalf = parent2Array[i][0:(crossOverIndex-1)]
    parent2SecondHalf = parent2Array[i][(crossOverIndex-1):(parent2.shape[1])]

    indicesListNotMatching1stParent = []
    indicesListNotMatching2ndParent = []
    length2ndHalf = len(parent2SecondHalf)

    parent1OriginalSecondHalfCopy = parent1SecondHalf.copy()
    parent1ResultSecondHalf = parent2SecondHalf.copy()

    parent2OriginalSecondHalfCopy = parent2SecondHalf.copy()
    parent2ResultSecondHalf = parent1SecondHalf.copy()

    for j in range(length2ndHalf):
      if parent1ResultSecondHalf[j] not in parent1OriginalSecondHalfCopy:
        indicesListNotMatching1stParent.append(j)
      else:
        resultParent1 = numpy.where(parent1OriginalSecondHalfCopy == parent1ResultSecondHalf[j])
        parent1OriginalSecondHalfCopy = numpy.delete(parent1OriginalSecondHalfCopy, resultParent1[0][0])

    for k in range(len(parent1OriginalSecondHalfCopy)):
      parent1ResultSecondHalf[indicesListNotMatching1stParent[k]] = parent1OriginalSecondHalfCopy[k]

    for l in range(length2ndHalf):
      if parent2ResultSecondHalf[l] not in parent2OriginalSecondHalfCopy:
        indicesListNotMatching2ndParent.append(l)
      else:
        resultParent2 = numpy.where(parent2OriginalSecondHalfCopy == parent2ResultSecondHalf[l])
        parent2OriginalSecondHalfCopy = numpy.delete(parent2OriginalSecondHalfCopy, resultParent2[0][0])

    for m in range(len(parent2OriginalSecondHalfCopy)):
      parent2ResultSecondHalf[indicesListNotMatching2ndParent[m]] = parent2OriginalSecondHalfCopy[m]

    recombinedParent1 = numpy.concatenate((parent1FirstHalf, parent1ResultSecondHalf))
    recombinedParent2 = numpy.concatenate((parent2FirstHalf, parent2ResultSecondHalf))

    recombinedParent1Array[i] = recombinedParent1
    recombinedParent2Array[i] = recombinedParent2

  recombinationResults.append(recombinedParent1Array)
  recombinationResults.append(recombinedParent2Array)

  return recombinationResults

In [13]:
def mutation(recombinationResults):
  randGen = random.randint(0, 5)
  numberMutationsWanted = 3

  if randGen > 3:
    print("Mutation")
    for i in range(numberMutationsWanted):
      randStartInd = random.randint(0, len(recombinationResults[0][0])-1)
      randEndInd = random.randint(0, len(recombinationResults[0][0])-1)

      for i in range(len(recombinationResults[0])):
        print("Start mutation at:  " + str(randStartInd))
        print("End mutation at:  " + str(randEndInd))
        startIndexValueParent1 = recombinationResults[0][i][randStartInd]
        recombinationResults[0][i][randStartInd] = recombinationResults[0][i][randEndInd]
        recombinationResults[0][i][randEndInd] = startIndexValueParent1

        startIndexValueParent2 = recombinationResults[1][i][randStartInd]
        recombinationResults[1][i][randStartInd] = recombinationResults[1][i][randEndInd]
        recombinationResults[1][i][randEndInd] = recombinationResults[1][i][randStartInd]

  else:
    print("No mutation")

  return recombinationResults

In [14]:
def evolutionary_algo(class_subject_hours_dataframe, teacher_subject_dataframe, singleTimetable, intialPopultationSize, num_generations):
  population = generate_intial_population(class_subject_hours_dataframe, teacher_subject_dataframe, singleTimetable, intialPopultationSize)

  for generation in range(num_generations):
    fitness = fitnessFunction(population)

    min_value = min(fitness)
    min_index = fitness.index(min_value)
    fitnessWithoutMin = fitness.copy()
    fitnessWithoutMin[min_index] = 1000
    second_min_value = min(fitnessWithoutMin)
    second_min_index = fitness.index(second_min_value)

    firstParent = population[min_index]
    secondParent =  population[second_min_index]

    print("Parent 1:  " + str(firstParent))
    print("Parent 2:  " + str(secondParent))

    recombinationParentResults = recombine(firstParent, secondParent)
    print("Recombination results:  " + str(recombinationParentResults))

    recombinationParentResultsMutation = mutation(recombinationParentResults)
    print("Mutation results:  " + str(recombinationParentResultsMutation))

    # make up new population
    population = []
    population.append(firstParent)
    population.append(secondParent)
    population.append(recombinationParentResults[0])
    population.append(recombinationParentResults[1])
    print("Population going in:  " + str(population))

    # The best result in the current iteration.
    fitness = fitnessFunction(population)
    min_value = min(fitness)
    min_index = fitness.index(min_value)
    print("Best result : " + str(population[min_index]))
    print("Number of clashes:  " + str(min_value))

# Execution

In [187]:
intial_population_size = 10
num_generations = 10

evolutionary_algo(class_subject_hours_dataframe, teacher_subject_dataframe, singleTimetable, intial_population_size, num_generations)

Parent 1:  [[('English', 'ProfessorA') None ('History', 'ProfessorB')
  ('Chemistry', 'ProfessorB') ('History', 'ProfessorB')
  ('Maths', 'ProfessorA') ('English', 'ProfessorA')
  ('Chemistry', 'ProfessorB') ('Chemistry', 'ProfessorB')
  ('Maths', 'ProfessorC') ('Chemistry', 'ProfessorB') None
  ('Maths', 'ProfessorC') ('English', 'ProfessorA')
  ('English', 'ProfessorA') ('Chemistry', 'ProfessorB')]
 [('History', 'ProfessorB') ('English', 'ProfessorA')
  ('History', 'ProfessorB') ('Maths', 'ProfessorA')
  ('Physics', 'ProfessorC') ('Physics', 'ProfessorC')
  ('History', 'ProfessorB') ('Maths', 'ProfessorA')
  ('English', 'ProfessorA') ('History', 'ProfessorB') None
  ('English', 'ProfessorA') None None ('History', 'ProfessorB')
  ('English', 'ProfessorA')]]
Parent 2:  [[('Chemistry', 'ProfessorB') ('Maths', 'ProfessorC')
  ('Maths', 'ProfessorA') ('English', 'ProfessorA')
  ('Chemistry', 'ProfessorB') None ('English', 'ProfessorA')
  ('Chemistry', 'ProfessorB') ('English', 'ProfessorA

/tmp/ipykernel_12580/2081026117.py:31: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if parent1ResultSecondHalf[j] not in parent1OriginalSecondHalfCopy:
/tmp/ipykernel_12580/2081026117.py:41: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if parent2ResultSecondHalf[l] not in parent2OriginalSecondHalfCopy:
